# Action class

In [1]:
class Action:
    pre = {} #preconditions
    eff={} #effects
    cost=0
    name = ""
    def __init__(self, p, e, c, n):
        self.pre = p
        self.eff = e
        self.name = n
        self.cost = c
        
    def formatFluents(self, d):
        string = ""
        for k in d.keys():
            if(d[k]==0):
                string += "\n- " + k
            else:
                string += "\n+ " + k
        return string

    def __str__(self):
        return f'Name: {self.name} \nCost: {self.cost} \nPreconditions: {self.formatFluents(self.pre)} \nEffects: {self.formatFluents(self.eff)}'

## Define actions

In [2]:
a1 = Action(p={"person":1, "hungry":0, "hasCake":1},e={"hasCake":0, "person":0},c=1,n="EatCake B")
a2 = Action(p={"person":1, "hungry":1, "hasCake":1},e={"hasCake":0, "eatenCake":1,"hungry":0},c=1,n="EatCake A")
a3 = Action(p={"person":1,"hasMix":1},e={"hasCake":1,"hasMix":0},c=1,n="MakeCake")
a4 = Action(p={"person":1,"hungry":0},e={"hungry":1},c=1,n="Wait")
a5 = Action(p={"person":1},e={"hasMix":1},c=1,n="Go Shopping")

actions = [a1,a2,a3,a4,a5]

## Define start and goal state

In [3]:
goal = {'person':1,'hungry':0, "hasCake":1, "eatenCake":1}
start = {'person':1}

## Define functions

In [4]:
# Get only positive fluents
def getPositiveFluents(state):
    return {key:val for key, val in state.items() if val == 1}

# Get only negative fluents
def getNegativeFluents(state):
    return {key:val for key, val in state.items() if val == 0}

# Check if two states match (match positive fluents only)
def statesMatch(s1,s2):
    s1_pos = getPositiveFluents(s1).copy()
    s2_pos = getPositiveFluents(s2).copy()
    return s1_pos == s2_pos

# Checks if an action is applicable for a given state
def isApplicable(state, action):
    positives = getPositiveFluents(state).copy()
    positivePres = getPositiveFluents(action.pre).copy()
    negativePres = getNegativeFluents(action.pre).copy()
    positivesFulfilled = positivePres.items() <= positives.items()
    negativesNotConsumed = any(p in positives for p in negativePres) == False
    return positivesFulfilled & negativesNotConsumed
    

# For a state and all possible actions, return the [applicable actions] for the given state
def applicableActions(state, actions):
    arr = []
    for a in actions:
        if isApplicable(state, a):
            arr.append(a)
    return arr

# Given a state and an action, applies the action and returns the following state if all preconditions are met
def applyAction(state, action):    
    if isApplicable(state, action) == False:
        raise Exception(f'Action {action.name} cannot be applied to state {str(state)}')
    newState = state.copy()
    effects = action.eff.copy()
    for k in effects.keys():
        newState[k] = effects[k]
        if effects[k] == 0:
            newState.pop(k)
    return newState

# A*

In [5]:
# Node class for frontier
class Node:
    
    def __init__(self, state, cost, path):
        self.state = state
        self.cost = cost
        self.path = path

    def __str__(self):
        pathNames = [p.name for p in self.path]
        pathStr = '[' + ', '.join(pathNames) + ']'
        return f'State: {self.state}, Cost: {self.cost}, Path: {pathStr}'

In [6]:
# Heuristic function for A* – a distance heuristic based on the amount of fluents that differ from the goal
def heuristic(nextState, goal):
    return len(dict(set(goal.items()) - set(nextState.items())))

In [7]:
#Input: Start, Goal, Actions. Output: Path of Actions
def aStar(start, goal, actions):

    # Init start node, frontier and visited list
    init = Node(start, 0, [])
    frontier = [init]
    visited = [init]

    while True:

        # Get first node from frontier
        head = frontier.pop(0)

        state = head.state.copy()

        # When A* has found a path, return it
        if (statesMatch(state, goal)):
            return head.path

        applicActions = applicableActions(state, actions)

        for action in applicActions:
            nextState = applyAction(state, action).copy()

            # Only apply action if we haven't visited the state before
            vis = False
            for v in visited:
                if v.state == nextState:
                    vis = True
            if vis == False:
                newPath = head.path + [action]
                cost = heuristic(nextState, goal) + head.cost + action.cost
                newNode = Node(nextState, cost, newPath)
                visited.append(newNode)
                frontier.append(newNode)

        # Sort frontier based on costs
        frontier = sorted(frontier, key=lambda x:x.cost)
        

In [8]:
print([a.name for a in aStar(start, goal, actions)])

['Go Shopping', 'MakeCake', 'Wait', 'EatCake A', 'Go Shopping', 'MakeCake']


# Testing

In [9]:
def testStatesMatch():
    
    # Test null state
    s1 = {}
    s2 = {}
    assert(statesMatch(s1,s2))
    
    # Test equal state
    s1 = {'person':1,'hungry':0}
    s2 = {'person':1,'hungry':0}
    assert(statesMatch(s1,s2))
    
    # Test pass by omission
    s1 = {'person':1}
    s2 = {'person':1, 'hungry':0}
    assert(statesMatch(s1,s2))
    
    # Test non-equal fluents state
    s1 = {'person':1,'hungry':0}
    s2 = {'person':0,'hungry':1}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'hungry':1}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'hungry':0}
    assert(statesMatch(s1,s2) == False)
    
    # Test different states
    s1 = {'person':1}
    s2 = {'person':1, 'hungry':1}
    assert(statesMatch(s1,s2) == False)
    
    print(f'All test cases for function "statesMatch" passed')

    
def testGetPositiveFluents():
    
    s = {'person':0,'hungry':0}
    assert(getPositiveFluents(s) == {})
    
    s = {'person':1,'hungry':0}
    assert(getPositiveFluents(s) == {'person':1})
    
    s = {'person':1, 'hungry':1, 'angry': 1}
    assert(getPositiveFluents(s) == {'person':1, 'hungry':1, 'angry': 1})
    
    s = {'person':1}
    assert(getPositiveFluents(s) == {'person':1})
    
    print(f'All test cases for function "getPositiveFluents" passed')
    
    
def testApplicableActions():
    # Test cases from powerpoint examples
    start = {'person':1}
    applActions = applicableActions(start, actions)
    assert(all(a in [a4,a5] for a in applActions))
    assert(any(a in [a1,a2,a3] for a in applActions) == False)
    
    s1 = {'person': 1, 'hasMix': 1}
    applActions = applicableActions(s1, actions)
    assert(all(a in [a3,a4,a5] for a in applActions))
    assert(any(a in [a1,a2] for a in applActions) == False)
    
    s2 = {'person': 1, 'hungry': 1}
    applActions = applicableActions(s2, actions)
    assert(all(a in [a5] for a in applActions))
    assert(any(a in [a1,a2,a3,a4] for a in applActions) == False)
    
    s3 = {'person': 1, 'hasCake': 1}
    applActions = applicableActions(s3, actions)
    assert(all(a in [a1,a4,a5] for a in applActions))
    assert(any(a in [a2,a3] for a in applActions) == False)
    
    s4 = {'person': 1, 'hungry': 1, 'hasMix': 1}
    applActions = applicableActions(s4, actions)
    assert(all(a in [a3,a5] for a in applActions))
    assert(any(a in [a1,a2,a4] for a in applActions) == False)
    
    print(f'All test cases for function "applicableActions" passed')

    
def testApplyAction():
    
    # Test added fluents
    s = {'a':1}
    a = Action(p={'a':1},e={'b':1},c=0,n="")
    assert(applyAction(s, a) == {'a':1, 'b':1})
    a = Action(p={'a':1},e={'b':1, 'c':1},c=0,n="")
    assert(applyAction(s, a) == {'a':1, 'b':1, 'c':1})
    
    # Test omission
    s = {'a':1}
    a = Action(p={'a':1, 'c':0},e={'b':1, 'c':1},c=0,n="")
    assert(applyAction(s, a) == {'a':1, 'b':1, 'c':1})
    
    # Test negative (applyAction should throw exception)
    try:
        s = {'a':1, 'c':1}
        a = Action(p={'a':1, 'c':0},e={'b':1, 'c':1},c=0,n="")
        raise Exception(f'Test negative failed')
    except:
        pass
    
    
    print(f'All test cases for function "applyAction" passed')


# Tests valid solutions to the cake-PDDL (See slide 63 - PDDL lecture)
def testPDDLsolutions():
    
    # Test valid path from start to goal
    s1 = applyAction(start, a5)
    s3 = applyAction(s1, a3)
    s5 = applyAction(s3, a5)
    s9 = applyAction(s5, a4)
    s11 = applyAction(s9, a2)
    g = applyAction(s11, a3)
    assert(statesMatch(g, goal))
    
    # Test another valid path from start to goal
    s2 = applyAction(start, a4)
    s4 = applyAction(s2, a5)
    s7 = applyAction(s4, a3)
    s10 = applyAction(s7, a2)
    s11_v2 = applyAction(s10, a5)
    
    # sanity check - check that s11 from both versions are identical
    assert(statesMatch(s11, s11_v2)) 
    
    g2 = applyAction(s11_v2, a3)
    assert(statesMatch(g2, goal))
    

    print(f'All test cases for function "testPDDLsolutions" passed')

    
# Validate a PDDL solution
def validatePDDL(start, goal, solution):
    s = start
    for a in solution:
        s = applyAction(s, a)
    assert(statesMatch(s, goal))
    print(f'The given PDDL solution is valid')

    
# Test heuristic function
def testHeuristic():
    
    d1 = {'a':1}
    d2 = {'a':1, 'b':2}
    assert(heuristic(d1,d2) == 1)
    
    d1 = {'a':0}
    d2 = {'a':1, 'b':2}
    assert(heuristic(d1,d2) == 2)
    
    d1 = {'a':1, 'b':2}
    d2 = {'a':1, 'b':2}
    assert(heuristic(d1,d2) == 0)
    
    d1 = {'b':2, 'a':1}
    d2 = {'a':1, 'b':2}
    assert(heuristic(d1,d2) == 0)
    
    d1 = {'b':2, 'a':1}
    d2 = {}
    assert(heuristic(d1,d2) == 0)
    
    d1 = {'b':2, 'a':1}
    d2 = {'a':1}
    assert(heuristic(d1,d2) == 0)

    print(f'All test cases for function "heuristic" passed')

In [10]:
testStatesMatch()
testGetPositiveFluents()
testApplicableActions()
testApplyAction()
testPDDLsolutions()
testHeuristic()

All test cases for function "statesMatch" passed
All test cases for function "getPositiveFluents" passed
All test cases for function "applicableActions" passed
All test cases for function "applyAction" passed
All test cases for function "testPDDLsolutions" passed
All test cases for function "heuristic" passed


# Define our PDDL

In [11]:
goal = {'atDestination':1,'haveBags':1} #I believe i dont need to put hungry 0
start = {'hungry':1} #world-comp: any positive fluent not specified means negated[...]
a1 = Action(p={},e={"rested":1},c=2,n="Sleep")
a2 = Action(p={"haveMoney":1,"haveBags":0},e={"carChecked":1,"haveMoney":0},c=2,n="CheckCar")
a3 = Action(p={"carChecked":1,"haveBags":1,"rested":1,"carFuel":1},e={"rested":0,'carChecked':0,"carFuel":0,'atDestination':1},c=4,n="Drive")
a4 = Action(p={"haveMoney":1},e={"haveFood":1,"haveMoney":0},c=1,n="BuyFood")
a5 = Action(p={"hungry":1,"haveFood":1},e={"hungry":0,"haveFood":0},c=1,n="Eat")
a6 = Action(p={"haveMoney":1},e={"haveMoney":0,"carFuel":1},c=1,n="FuelUpCar")
a7 = Action(p={},e={"haveMoney":1},c=1,n="WithdrawMoney")
a8 = Action(p={},e={"haveBags":1,},c=1,n="LoadCar")
actions=[a1,a2,a3,a4,a5,a6,a7,a8]

## Test A* on our PDDL and validate the solution

In [12]:
solution = aStar(start, goal, actions)
print(f'Solution: {[a.name for a in solution]}')
validatePDDL(start, goal, solution)

Solution: ['WithdrawMoney', 'CheckCar', 'LoadCar', 'WithdrawMoney', 'BuyFood', 'Eat', 'WithdrawMoney', 'FuelUpCar', 'Sleep', 'Drive']
The given PDDL solution is valid
